# Final Submission Version

In [ ]:
# Imports and Setup
import os
import sys
from argparse import Namespace
import torch

import train
import utils
import config

print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
print(f"Using device: {config.DEVICE}")

## 1. Single Method

### 1.1. LoRA

### 1.2. LoHa

### 1.3. LoKr

### 1.4. AdaLoRA

In [ ]:
from argparse import Namespace

args = Namespace(
    output_dir="results_adalora_qvd_r4-6_a2_lr2e-4",
    seed=42,
    peft_method="adalora",
    target_modules=["query", "value", "attention.output.dense"],
    lora_r=4,
    lora_alpha=2,
    lora_dropout=0.1,
    adalora_init_r=6,
    adalora_tinit=0,
    adalora_tfinal=0,
    adalora_deltaT=1,
    adalora_beta1=0.85,
    adalora_beta2=0.85,
    learning_rate=2e-4,
    num_train_epochs=3,
    train_batch_size=128,
    eval_batch_size=128,
    optimizer="adamw_torch"
)

os.makedirs(args.output_dir, exist_ok=True)

print("Parameters:")
for k, v in vars(args).items():
    print(f"  {k}: {v}")


In [ ]:
# launch training
print("=== START TRAINING ===")
final_accuracy = train.main_train(args)
print("=== TRAINING FINISHED ===")